In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
os.chdir('/content/drive/MyDrive/')

In [ ]:
import pandas as pd
import numpy as np

df = pd.read_csv('/content/drive/MyDrive/D210/churn_clean.csv', index_col=0)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10000 entries, 1 to 10000
Data columns (total 49 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Customer_id           10000 non-null  object 
 1   Interaction           10000 non-null  object 
 2   UID                   10000 non-null  object 
 3   City                  10000 non-null  object 
 4   State                 10000 non-null  object 
 5   County                10000 non-null  object 
 6   Zip                   10000 non-null  int64  
 7   Lat                   10000 non-null  float64
 8   Lng                   10000 non-null  float64
 9   Population            10000 non-null  int64  
 10  Area                  10000 non-null  object 
 11  TimeZone              10000 non-null  object 
 12  Job                   10000 non-null  object 
 13  Children              10000 non-null  int64  
 14  Age                   10000 non-null  int64  
 15  Income                10

In [ ]:
#rename columns to python casing.
col_head = {
    'CaseOrder': 'case_order',
    'Customer_id': 'customer_id',
    'Interaction': 'interaction',
    'UID': 'uid',
    'City': 'city',
    'State': 'state',
    'County': 'county',
    'Zip': 'zip',
    'Lat': 'lat',
    'Lng': 'lng',
    'Population': 'population',
    'Area': 'area',
    'TimeZone': 'timezone',
    'Job': 'job',
#renaming the children column entirely to make sense for tableu analysis
    'Children': 'dependent',
    'Age': 'age',
    'Education': 'education',
    'Employment': 'employment_type',
    'Income': 'income',
#Renaming marital to married
    'Marital': 'married',
    'Gender': 'gender',
    'Churn': 'churn',
    'Outage_sec_perweek': 'outage_sec_perweek',
    'Email': 'email',
    'Contacts': 'contacts',
    'Yearly_equip_failure': 'yearly_equip_failure',
    'Techie': 'techie',
    'Contract': 'contract',
    'Port_modem': 'port_modem',
    'Tablet': 'tablet',
    'InternetService': 'internet_service',
    'Phone': 'phone',
    'Multiple': 'multiple_lines',
    'OnlineSecurity': 'online_security',
    'OnlineBackup': 'online_backup',
    'DeviceProtection': 'device_protection',
    'TechSupport': 'tech_support',
    'StreamingTV': 'streaming_tv',
    'StreamingMovies': 'streaming_movies',
    'PaperlessBilling': 'paperless_billing',
    'PaymentMethod': 'payment_method',
    'Tenure': 'tenure',
    'MonthlyCharge': 'monthly_charge',
    'Bandwidth_GB_Year': 'bandwidth_gb_year',
    'Item1': 'response_timeliness',
    'Item2': 'fix_timeliness',
    'Item3': 'replacement_timeliness',
    'Item4': 'service_reliability',
    'Item5': 'service_options',
    'Item6': 'respectful_communication',
    'Item7': 'courteous_interaction',
    'Item8': 'active_listening_skill'}

df.rename(columns=col_head, inplace=True)

#Reexpressing the values of payment method to be more concise.
dict_pay = {"payment_method": {
     "Credit Card (automatic)": "Autopay",
     "Bank Transfer(automatic)": "Autopay",
     "Electronic Check": "eCheck",
     "Mailed Check": "Check",
}}
df.replace(dict_pay, inplace = True)

#reexpressing the values of marital_type to be yes/no answers for current marriage status
dict_mar = {"married": {
     "Never Married": "No",
     "Married": "Yes",
     "Single": "No",
     "Divorced": "No",
     "Widowed": "No",
     "Separated": "No"
}}
df.replace(dict_mar, inplace = True)

#reexpressing the value of dependant column (object) into yes/no values where any amount greater than 0 = yes
def convert_to_yes_no(value):
    if value == 0:
        return 'No'
    else:
        return 'Yes'

df['dependent'] = df['dependent'].apply(convert_to_yes_no)

#Change 'contract' values to match that of other data set
dict_con = {"contract":{
    "Two Year": "Two year"
}}
df.replace(dict_con, inplace=True)

#Changing columns into category data type
df_cat = ['area', 'married', 'gender', 'payment_method', 'response_timeliness', 'fix_timeliness',
          'replacement_timeliness', 'service_reliability', 'service_options', 'respectful_communication',
          'courteous_interaction', 'active_listening_skill']

df[df_cat] = df[df_cat].astype('category')

#rounding monthly_charge column and rounding tenure column
df['monthly_charge'] = df['monthly_charge'].round(2)
df['tenure'] = df['tenure'].round(0)

#The 'internet_service' column does not contain null or missing values instead "None" is interpreted as a null value. To fix this we change "None" to "none"
for col in ['internet_service']:
    df[col].fillna('None', inplace=True)

df.head()

,customer_id,interaction,uid,city,state,county,zip,lat,lng,population,...,monthly_charge,bandwidth_gb_year,response_timeliness,fix_timeliness,replacement_timeliness,service_reliability,service_options,respectful_communication,courteous_interaction,active_listening_skill
CaseOrder,,,,,,,,,,,,,,,,,,,,,
1,K409198,aa90260b-4141-4a24-8e36-b04ce1f4f77b,e885b299883d4f9fb18e39c75155d990,Point Baker,AK,Prince of Wales-Hyder,99927,56.25100,-133.37571,38,...,172.46,904.536110,5,5,5,3,4,4,3,4
2,S120509,fb76459f-c047-4a9d-8af9-e0f7d4ac2524,f2de8bef964785f41a2959829830fb8a,West Branch,MI,Ogemaw,48661,44.32893,-84.24080,10446,...,242.63,800.982766,3,4,3,3,4,3,4,4
3,K191035,344d114c-3736-4be5-98f7-c72c281e2d35,f1784cfa9f6d92ae816197eb175d3c71,Yamhill,OR,Yamhill,97148,45.35589,-123.24657,3735,...,159.95,2054.706961,4,4,2,4,4,3,3,3
4,D90850,abfa2b40-2d43-4994-b15a-989b8c79e311,dc8a365077241bb5cd5ccd305136b05e,Del Mar,CA,San Diego,92014,32.96687,-117.24798,13863,...,119.96,2164.579412,4,4,4,2,5,4,3,3
5,K662701,68a861fd-0d20-4e51-a587-8a90407ee574,aabb64a116e83fdc4befc1fbab1663f9,Needville,TX,Fort Bend,77461,29.38012,-95.80673,11352,...,149.95,271.493436,4,4,4,3,4,4,4,5


In [ ]:
#adding column to dataframe for distinction in tableau
df['dataset'] = 'wgu telecom'

In [ ]:
#cleaning dataset from Kaggle
kaggle_df = pd.read_csv('/content/drive/MyDrive/D210/WA_Fn-UseC_-Telco-Customer-Churn.csv')
kaggle_df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [ ]:
kaggle_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   gender            7043 non-null   object 
 2   SeniorCitizen     7043 non-null   int64  
 3   Partner           7043 non-null   object 
 4   Dependents        7043 non-null   object 
 5   tenure            7043 non-null   int64  
 6   PhoneService      7043 non-null   object 
 7   MultipleLines     7043 non-null   object 
 8   InternetService   7043 non-null   object 
 9   OnlineSecurity    7043 non-null   object 
 10  OnlineBackup      7043 non-null   object 
 11  DeviceProtection  7043 non-null   object 
 12  TechSupport       7043 non-null   object 
 13  StreamingTV       7043 non-null   object 
 14  StreamingMovies   7043 non-null   object 
 15  Contract          7043 non-null   object 
 16  PaperlessBilling  7043 non-null   object 


In [ ]:
# Renaming columns to Python casing
col_head = {
    'customerID': 'customer_id',
    'SeniorCitizen': 'senior_citizen',
    'Partner': 'married',
#keeping same name as churn dataset for married and dependent columns
    'Dependents': 'dependent',
    'PhoneService': 'phone',
    'MultipleLines': 'multiple_lines',
    'OnlineSecurity': 'online_security',
    'OnlineBackup': 'online_backup',
    'DeviceProtection': 'device_protection',
    'TechSupport': 'tech_support',
    'StreamingTV': 'streaming_tv',
    'StreamingMovies': 'streaming_movies',
    'PaperlessBilling': 'paperless_billing',
    'PaymentMethod': 'payment_method',
    'Contract': 'contract',
    'Churn': 'churn',
    'InternetService': 'internet_service',
    'TotalCharges': 'total_charges',
    'MonthlyCharges': 'monthly_charge',
    'gender': 'gender'
}

# Renaming columns in the DataFrame
kaggle_df.rename(columns=col_head, inplace=True)

# Reexpressing the values of payment method to be more concise
kaggle_pay = {
    "payment_method": {
        "Credit card (automatic)": "Autopay",
        "Bank transfer (automatic)": "Autopay",
        "Electronic check": "eCheck",
        "Mailed check": "Check",
    }
}
kaggle_df.replace(kaggle_pay, inplace=True)

# Reexpressing the values of internet_service to be more concise
kaggle_int = {
    "internet_service": {
        "No": "None",
        "Fiber optic": "Fiber Optic"
    }
}
kaggle_df.replace(kaggle_int, inplace=True)

kaggle_no_internet = {
    "online_security": {"No internet service": "No"},
    "online_backup": {"No internet service": "No"},
    "device_protection": {"No internet service": "No"},
    "tech_support": {"No internet service": "No"},
    "streaming_tv": {"No internet service": "No"},
    "streaming_movies": {"No internet service": "No"}
}

# Renaming columns in the DataFrame
kaggle_df.replace(kaggle_no_internet, inplace=True)

# Reexpressing the values of gender to be more concise
kaggle_lines = {
    "multiple_lines": {
        "No phone service": "No",
    }
}
kaggle_df.replace(kaggle_lines, inplace=True)

# Displaying the modified DataFrame
kaggle_df.head()

,customer_id,gender,senior_citizen,married,dependent,tenure,phone,multiple_lines,internet_service,online_security,...,device_protection,tech_support,streaming_tv,streaming_movies,contract,paperless_billing,payment_method,monthly_charge,total_charges,churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No,DSL,No,...,No,No,No,No,Month-to-month,Yes,eCheck,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No,DSL,Yes,...,Yes,Yes,No,No,One year,No,Autopay,42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber Optic,No,...,No,No,No,No,Month-to-month,Yes,eCheck,70.70,151.65,Yes


In [ ]:
#column for distinction in tableau
kaggle_df['dataset'] = 'Competitor'

In [ ]:
# dropping columns not relevant to analysis (Kaggle)
kaggle_df = kaggle_df.drop(columns=['senior_citizen','total_charges', 'customer_id'])

# dropping columns not relevant to analysis (churn_clean)
df = df.drop(columns=['interaction', 'uid', 'timezone', 'job', 'age', 'income',
                       'outage_sec_perweek', 'email','contacts', 'yearly_equip_failure',
                       'techie', 'port_modem','tablet','bandwidth_gb_year','response_timeliness',
                       'fix_timeliness','replacement_timeliness', 'service_reliability', 'service_options',
                       'respectful_communication', 'courteous_interaction', 'active_listening_skill',
                       'customer_id', 'city', 'state', 'county', 'zip', 'lat', 'lng','population', 'area'])

In [ ]:
combined_df = pd.concat([df, kaggle_df], ignore_index=True)

#saving csv to my folder
with open('/content/drive/MyDrive/D210/combined_df.csv', 'w') as f:
    combined_df.to_csv(f, index=False)